# Kaggle study day 18

Transfer Learning with VGG-16 CNN+AUG LB 0.1712 : https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712

### TL;DR

GPU에서 실행한다. CPU와는 호환성 문제가 있다.

1. 딥러닝의 하이퍼파라미터는 여러가지가 있으며, 이를 조정하려면 오래 걸린다(몇 주/달). 일반적으로 연구자들은 이 튜닝을 하고 다른 아키텍쳐보다 성능이 높은 아키텍쳐 세트를 발견하면 논문을 발표한다.
2. 이 모델은 사전 학습을 했기 때문에 빠르게 수렴되고 사용자도 CPU를 사용해야한다. 일부 라이브러리 문제로 CPU에선 작동하지 않는다.
3. 목적을 위해, 연구원들에 의해 이용할 수 있는 아키텍쳐를 사용할 수 있다.
4. 이미 변수 추출 방법을 '알고'있는 레이어인 pertrained nets을 사용하면, 하이퍼파라미터를 튜닝하지 않아도 된다. 이미 일부 데이터셋을 학습했기 때문에(imagenet), 사전 학습된 가중치는 좋은 가중치 초기화를 제공하고 이로 인해 Convnet은 빠르게 수렴된다(그렇지 않으면 이 심층 아키텍쳐에 며칠이 걸릴 수 있다). 예시로, VGG16, InceptionNet, googlenet, Resnet 등이 있다.
5. 이 커널에서는, 작은 크기의 이미지에서 매우 우수한 성능을 보이는 사전 학습된 VGG-16을 사용한다.

VGG 아키텍처는 작은 크기의 이미지(CIFAR-10)에서 잘 작동한다. 이 데이터셋에서도 잘 작동할 것이다. 이 코드엔 데이터 augmentation 단계도 포함되어 있으므로 성능이 상당히 개선된다.
(### augmentation : 원래 데이터를 부풀려서 성능 향상시킴)

In [1]:
# Mandatory imports 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10,10
%matplotlib inline

In [2]:
train = pd.read_json('../input/train.json')
target_train = train['is_iceberg']
test = pd.read_json('../input/test.json')

Keras는 사전학습된 VGG 구현을 제공한다. 라이브러리에 있으므로 직접 네트워크를 구축할 필요가 없다. 여기서 VGG의 마지막 레이어를 제거하고 이진예측을 위해 시그모이드 레이어를 넣을 것이다.

In [3]:
train['inc_angle']

0       43.9239
1       38.1562
2       45.2859
3       43.8306
4       35.6256
         ...   
1599         na
1600         na
1601         na
1602         na
1603         na
Name: inc_angle, Length: 1604, dtype: object

In [9]:
target_train = train['is_iceberg']
### inc_angle은 위에서 보이듯 object 타입이므로 numeric으로 변환
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle'] = train['inc_angle'].fillna(method='pad')
X_angle = train['inc_angle']
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle = test['inc_angle']

# train 데이터 생성
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1 + X_band_2) / 2
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([X_band_1[:,:,:,np.newaxis], 
                          X_band_2[:,:,:,np.newaxis], 
                          X_band_3[:,:,:,np.newaxis]], axis=-1)

X_band_test_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3 = (X_band_test_1 + X_band_test_2) / 2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:,:,:,np.newaxis]
                          , X_band_test_2[:,:,:,np.newaxis]
                         , X_band_test_3[:,:,:,np.newaxis]], axis=-1)

# import Keras
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

# Data Aug for multi-input
batch_size = 64
# 이미지 변환 정의
gen = ImageDataGenerator(horizontal_flip = True, vertical_flip = True, width_shift_range = 0., height_shift_range = 0.,
                         channel_shift_range=0, zoom_range = 0.2, rotation_range = 10)

# 두 생성자를 병합하는 함수
# 동일한 랜덤 시드를 가진 동일한 생성자를 y와 angle 배열에 모두 사용
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()
        # 배열이 동일하다고 가정 - 마음의 평화를 위한 것이지만 학습속도를 늦춘다
        # np.testing.assert_array_equal(X1i[0],X2i[0])
        yield [X1i[0], X2i[1]], X1i[1]
        
    
# 최종적으로 생성자 생성
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=10, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

def getVggAngleModel():
    input_2 = Input(shape=[1], name='angle')
    angle_layer = Dense(1,)(input_2)
    base_model = VGG16(weights='imagenet', include_top=False,
                      input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    
    x = GlobalMaxPooling2D()(x)  ### GlobalMaxPooling : 차원을 급격히 줄임. 각 레이어에서 Max값 하나만 추출
    merge_one = concatenate([x, angle_layer])
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model([base_model.input, input_2], predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                 optimizer=sgd, metrics=['accuracy'])
    
    return model


# Data Augmentation과 함께 K-fold Cross Validation 사용
def myAngleCV(X_train, X_angle, X_test):
    K = 3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log = 0
    y_valid_pred_log = 0.0 * target_train
    
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n==================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout = target_train[test_idx]
        
        # Angle
        X_angle_cv = X_angle[train_idx]
        X_angle_hold = X_angle[test_idx]
        
        # 파일 경로 정의 및 callbacks 
        file_path = '%s_aug_model_weights.hdf5'%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel = getVggAngleModel()
        galaxyModel.fit_generator(gen_flow,
                                 steps_per_epoch=24, epochs=100, shuffle=True, verbose=1,
                                 validation_data=([X_holdout, X_angle_hold], Y_holdout), callbacks=callbacks)
        
        # 최적 모델 얻기
        galaxyModel.load_weights(filepath=file_path)
        # train score
        score = galaxyModel.evaluate([X_train_cv, X_angle_cv], y_train_cv, verbose=0)
        print('Train loss :', score[0])
        print('Train accuracy :', score[1])
        # test score
        score = galaxyModel.evaluate([X_holdout, X_angle_hold], Y_holdout, verbose=0)
        print('Test loss :', score[0])
        print('Test accuracy :', score[1])
        # Validation score
        pred_valid = galaxyModel.predict([X_holdout, X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])
        
        # test scores
        temp_test = galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log += temp_test.reshape(temp_test.shape[0])
        # train scores
        temp_train = galaxyModel.predict([X_train, X_angle])
        y_train_pred_log += temp_train.reshape(temp_train.shape[0])
        
    y_test_pred_log = y_test_pred_log / K
    y_train_pred_log = y_train_pred_log / K
    
    print('\n Train Log Loss Validation = ', log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation = ', log_loss(target_train, y_valid_pred_log))
    
    return y_test_pred_log

In [10]:
preds = myAngleCV(X_train, X_angle, X_test)


==================FOLD= 0


c:\anaconda3\envs\study\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
24/24 [==============================] - 266s 11s/step - loss: 1.7824 - accuracy: 0.5000 - val_loss: 0.7628 - val_accuracy: 0.5308
Epoch 2/100
24/24 [==============================] - 278s 11s/step - loss: 0.8393 - accuracy: 0.5274 - val_loss: 0.7070 - val_accuracy: 0.5308
Epoch 3/100
24/24 [==============================] - 285s 12s/step - loss: 0.7662 - accuracy: 0.5169 - val_loss: 0.6605 - val_accuracy: 0.5308
Epoch 4/100
24/24 [==============================] - 284s 12s/step - loss: 0.6653 - accuracy: 0.5996 - val_loss: 0.5342 - val_accuracy: 0.6841
Epoch 5/100
24/24 [==============================] - 292s 12s/step - loss: 0.5603 - accuracy: 0.6445 - val_loss: 0.4830 - val_accuracy: 0.8056
Epoch 6/100
24/24 [==============================] - 294s 12s/step - loss: 0.4761 - accuracy: 0.7587 - val_loss: 0.4008 - val_accuracy: 0.8262
Epoch 7/100
24/24 [==============================] - 298s 13s/step - loss: 0.4594 - accuracy: 0.7964 - val_loss: 0.3900 - val_accuracy: 0.8486

Epoch 2/100
24/24 [==============================] - 300s 13s/step - loss: 0.5937 - accuracy: 0.6733 - val_loss: 0.3420 - val_accuracy: 0.8446
Epoch 3/100
24/24 [==============================] - 294s 12s/step - loss: 0.3684 - accuracy: 0.8341 - val_loss: 0.3559 - val_accuracy: 0.8071
Epoch 4/100
24/24 [==============================] - 288s 12s/step - loss: 0.3276 - accuracy: 0.8458 - val_loss: 0.2879 - val_accuracy: 0.8614
Epoch 5/100
24/24 [==============================] - 292s 12s/step - loss: 0.2897 - accuracy: 0.8605 - val_loss: 0.2784 - val_accuracy: 0.8764
Epoch 6/100
24/24 [==============================] - 296s 12s/step - loss: 0.2515 - accuracy: 0.8913 - val_loss: 0.2816 - val_accuracy: 0.8745
Epoch 7/100
24/24 [==============================] - 300s 12s/step - loss: 0.2233 - accuracy: 0.8996 - val_loss: 0.2635 - val_accuracy: 0.8820
Epoch 8/100
24/24 [==============================] - 291s 12s/step - loss: 0.2324 - accuracy: 0.9005 - val_loss: 0.3246 - val_accuracy: 0.8577

In [11]:
submission = pd.DataFrame()
submission['id'] = test['id']
submission['is_iceberg'] = preds
submission.to_csv('sub.csv', index=False)